![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to capture video from a V4L2 device and display it inside this jupyter notebook using the GStreamer multimedia framework.

Four types of V4L2 devices are supported in this notebook:
* Virtual Video Test driver (vivid)
* USB Video Class (UVC) driver (usbcam)
* MIPI CSI-2 capture pipeline using the Leopard IMX274 FMC daughter card (mipi)
* HDMI Rx capture pipeline (hdmi)

Vivid (https://www.kernel.org/doc/html/latest/media/v4l-drivers/vivid.html) is configured to emulate a USB webcam in software and hence does not require any additional hardware to run this notebook. By default, the notebook is configured to use vivid.

The UVC driver (https://www.kernel.org/doc/html/latest/media/v4l-drivers/uvcvideo.html) is commonly used to capture video from a USB webcam.

The Leopard IMX274 FMC daughter card (https://leopardimaging.com/product/csi-2-mipi-modules-i-pex/li-imx274mipi-fmc/) can be used to capture video through a MIPI CSI-2 interface. The MIPI CSI-2 capture pipeline is implemented inside the PL and includes a basic ISP.

The HDMI Rx capture pipeline is implemented inside the PL.

The video pipeline is composed of the following GStreamer elements:
* The ``mediasrcbin`` element is used to capture video from a V4L2 device. It is a bin element on top of the standard ``v4l2src`` element which performs additional media pipeline initialization (if needed).
* The ``jpegenc`` element is used to compress the raw video format to JPEG.
* The ``appsink`` element is used to make the JPEG frames available to the jupyter notebook where they are displayed.

In this notebook, you will:
1. Create a GStreamer video pipeline that captures video from a V4L2 device and displays the video inside this notebook.
2. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import glob
import subprocess
import pydot
import sys
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

This is the Base TRD notebook 2 (nb2).

In [ ]:
nb = "nb2"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environment variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Enable debug by setting the debug string, set default to level 1 for all categories.

In [ ]:
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Create and Configure the GStreamer Elements

The ``get_media_by_device`` function returns the matching media node for a given video capture source. The following sources are supported in this notebook:
* ``vivid`` : virtual video device (default)
* ``usb`` : requires USB webcam
* ``mipi`` : platform1 only, requires FMC card
* ``hdmi`` : platform3 only, requires HDMI input

In [ ]:
def get_media_dev_by_name(src):
    sources = {
        'vivid' : 'vivid',
        "usb" : 'uvcvideo',
        'mipi' : 'vcap_csi',
        'hdmi' : 'vcap_hdmi'
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

Select the ``source`` based on available media devices for this platform. The default source is set to ``vivid``. Update the value next to the comment to select USB webcam MIPI single-sensor, or HDMI Rx if connected.

In [ ]:
source = "vivid" # Change source to vivid, usb, mipi , hdmi

media_device = get_media_dev_by_name(source) 
if media_device is None:
    raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered, and the correct platform is used.')

The source pads of the ``mediasrcbin`` element are created dynamically when it detects the incoming stream. The ``pad-added`` signal is emitted and this ``pad_added`` callback function is executed. It links the source pads of the mediasrcbin elements to the sink pads of the ``caps`` elements.

Set the ``io-mode`` to mmap on the pad which propagates to the ``v4l2src`` node.

In [ ]:
def pad_added(element, pad):
    sink_pad = caps.get_static_pad("sink")
    if not sink_pad.is_linked():
        pad.link(sink_pad)
        pad.set_property("io-mode", "mmap")

Create the ``mediasrcbin`` element which is a bin element that uses the standard ``v4l2src`` element inside. Set the following some properties:
* Set the ``media-device`` property to the desired media device node
* Register the above ``pad_added`` callback function with the ``pad-added`` signal of the ``mediasrcbin`` element.

In [ ]:
src = Gst.ElementFactory.make("mediasrcbin")
src.set_property("media-device", media_device)
src.connect("pad_added", pad_added);

Create a caps filter element to set the desired resolution (width and height) and format. The caps filter is configured to parse the mentioned properties from a string.

The default resolution is set to 1280x720 and the format to YUY2 as those are commonly supported by USB webcams.

In [ ]:
width = 1280
height = 720
fmt = "YUY2"

cap_string = "video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt
if source == "mipi" or source == "hdmi":
    fps = "60/1"
    cap_string = cap_string + ", framerate=" + fps

caps = Gst.ElementFactory.make("capsfilter")
cap = Gst.Caps.from_string(cap_string)
caps.set_property("caps", cap)

Create the ``jpegenc`` element to compress the YUY2 video frame to JPEG.

In [ ]:
jpegenc = Gst.ElementFactory.make("jpegenc")  

Create a callback function ``new_sample`` that retrieves the JPEG data from a GStreamer buffer object and passes it to the ``display`` function of the ``IPython.display`` module which displays the video frame inside the notebook.

In [ ]:
def new_sample(sink):   
    sample = sink.pull_sample()
    buffer = sample.get_buffer()
    ret, info = buffer.map(Gst.MapFlags.READ)
    
    display(Image(data=info.data))
    clear_output(wait=True)
    
    buffer.unmap(info)
    
    return Gst.FlowReturn.OK

Create the ``appsink`` element and set some properties:
* Set the ``drop`` property to ``True`` to drop old buffers when the buffer queue is full
* Set the ``max-buffers`` property to 0 to queue an unlimited number of buffers
* Set the ``emit-signals`` property to ``True`` to emit the ``new-sample`` signal

Register the above ``new_sample`` callback function with the ``new-sample`` signal of the ``appsink`` element.

In [ ]:
sink = Gst.ElementFactory.make("appsink")
sink.set_property("drop", True)
sink.set_property("max_buffers", 0)
sink.set_property("emit-signals", True)
sink.connect("new-sample", new_sample);

# Uncomment the below code to read back the newly set property values
#print("appsink properties: ")
#print("drop: " + str(sink.get_property("drop")))
#print("max_buffers: " + str(sink.get_property("max_buffers")))
#print("emit-signals: " + str(sink.get_property("emit-signals")))

# 4. Create and Run the GStreamer Pipeline

Create the pipeline, add all elements, and link them together.

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src)
pipeline.add(caps)
pipeline.add(jpegenc)
pipeline.add(sink)

caps.link(jpegenc)
jpegenc.link(sink);

The ``bus_call`` function listens on the bus for ``EOS`` and ``ERROR`` events. If any of these events occur, stop the pipeline (set to ``NULL`` state) and quit the main loop.

In case of an ``ERROR`` event, parse and print the error message.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video frames will be displayed below the following code cell. 

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 5. View the GStreamer Pipeline Graph

Register dot plugins for png export to work.

In [ ]:
!dot -c

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 6. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to capture video from a V4L2 device and play it back inside the jupyter notebook
2. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>